## delayed_task()

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
client.restart()

In [ ]:
import dask
import dask.dataframe as dd
from dask.distributed import Client
#client = Client('tcp://dask_scheduler:8786')

import time

@dask.delayed
def inc(x):
    time.sleep(.5)
    return x + 1

@dask.delayed
def dec(x):
    time.sleep(.5)
    return x - 1

@dask.delayed
def add(x, y):
    time.sleep(.5)
    return x + y

def delayed_task(N=10):
    zs = []
    for i in range(N):
        x = inc(i)
        y = dec(2*i)
        z = add(x, y)
        zs.append(z)
    zs = dask.persist(*zs)
    total = dask.delayed(sum)(zs) #computing with delayed
    #total = sum(zs)#.persist() #computing with futures
    total = total.persist()
    return total

In [ ]:
%%time
delayed_task()

## future_task()

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
client.restart()

In [ ]:
import numpy as np
import dask
import dask.dataframe as dd
from dask.distributed import Client
#client = Client('tcp://dask_scheduler:8786')

import time

def load(x):
    time.sleep(.1)
    return np.arange(100_000) + x

def process(x):
    time.sleep(.1)
    return x + 1

def future_task(N=10):
    futures = []
    for i in range(N):
        x = client.submit(load, i)
        y = client.submit(process, x)
        futures.append(y)
    result = [future.result() for future in futures]
    return futures, result

In [ ]:
future_task()

## r_csv()

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
client.restart()

In [ ]:
import dask
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client
#client = Client('tcp://dask_scheduler:8786')

def r_csv():
    #client = Client('tcp://dask_scheduler:8786')
    ddf = dd.read_csv('ToyData.csv')
    pddf = ddf.persist()
    sm = []
    for col in pddf.columns:
        sm.append(pddf[col].sum().persist())
        gsm = sum(sm)
        #gsm = dask.delayed(sum)(sm)
        finale = gsm.persist()
    return gsm, finale

In [ ]:
r_csv()